In [0]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

import nltk
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import string
import nltk.data

In [2]:
!ls

datalab  glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
drive	 glove.6B.200d.txt  glove.6B.50d.txt


In [16]:
df = pd.read_csv('drive/app/Misc/whatsapp/analysis/rajit_chat.txt', header=None,error_bad_lines=False,sep=']')

b'Skipping line 14807: expected 2 fields, saw 3\nSkipping line 16382: expected 2 fields, saw 3\nSkipping line 44293: expected 2 fields, saw 3\nSkipping line 44896: expected 2 fields, saw 3\nSkipping line 50313: expected 2 fields, saw 3\nSkipping line 52078: expected 2 fields, saw 3\nSkipping line 79430: expected 2 fields, saw 3\n'


In [17]:
df.head(2)

,0,1
0,"[30/01/14, 11:02:57 PM",‎Adamya Rajput changed the subject to “Richa ...
1,"[31/01/14, 3:09:54 AM",‎Adamya Rajput was added


In [0]:
# Removing [ from column 0
df[["trash","date_time"]] = pd.DataFrame(df[0].str.split('[',1).tolist())

In [19]:
df.head(2)

,0,1,trash,date_time
0,"[30/01/14, 11:02:57 PM",‎Adamya Rajput changed the subject to “Richa ...,,"30/01/14, 11:02:57 PM"
1,"[31/01/14, 3:09:54 AM",‎Adamya Rajput was added,,"31/01/14, 3:09:54 AM"


In [0]:
df = df.dropna()

In [0]:
# splitting name from column 1 to name
df[["name","text"]] = pd.DataFrame(df[1].str.split(':',1).tolist(),index=df.index)

In [22]:
df.head(2)

,0,1,trash,date_time,name,text
0,"[30/01/14, 11:02:57 PM",‎Adamya Rajput changed the subject to “Richa ...,,"30/01/14, 11:02:57 PM",‎Adamya Rajput changed the subject to “Richa ...,None
1,"[31/01/14, 3:09:54 AM",‎Adamya Rajput was added,,"31/01/14, 3:09:54 AM",‎Adamya Rajput was added,None


In [0]:
df = df.dropna()

In [0]:
df = df.drop([0,1,"trash"],axis = 1)

In [25]:
df.head(2)

,date_time,name,text
3,"30/01/14, 11:04:08 PM",Adamya Rajput,Pics bhejo bc exp 1 and 2 ki
9,"30/01/14, 11:05:45 PM",Bhandu,<‎image omitted>


In [0]:
# converting first column to datetime format
df['date_time'] =  pd.to_datetime(df['date_time'],errors = "coerce",dayfirst = True)

In [27]:
df.head(2)

,date_time,name,text
3,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki
9,2014-01-30 23:05:45,Bhandu,<‎image omitted>


In [0]:
# stripping name and text column to remove whitespaces from start and end

def strip(x):
  stripped = x.strip()
  return stripped

df['name'] = df[["name"]].apply(lambda x: strip(*x), axis=1)
df['text'] = df[["text"]].apply(lambda x: strip(*x), axis=1)

In [29]:
df.name.unique()

array(['Adamya Rajput', 'Bhandu', '\u202a+91\xa088265\xa089841\u202c',
       'rajit', 'Sukhman', 'Mayank', 'Suyash', 'SCOOBY',
       '\u202a+91\xa084472\xa005046\u202c', 'Sourabh Singhal',
       '\u202a+91 91 65 020604\u202c', 'Enzo', 'Shady', 'archu',
       'Shriram Yadav', 'MANDU GJU', 'Kallu',
       '\u202a+91\xa096542\xa092717\u202c', 'Meena',
       'The Billionaires Club 🤑',
       '\u200eSuyash changed the subject to “Our bitch', 'Raman Deora',
       '\u202a+971\xa050\xa0132\xa07301\u202c', 'sukhman Canada',
       'the complete digital marketing course* - https'], dtype=object)

In [30]:
# merging known names together and unknown names to value 1 to remove later

df['name'].replace('Sourabh Singhal', 'SCOOBY',inplace=True)
df['name'].replace('sukhman Canada', 'Sukhman',inplace=True)
df.replace({'name' : { '\u202a+91\xa088265\xa089841\u202c' : "Meena", '\u202a+91\xa084472\xa005046\u202c' : 1,
                      '\u202a+91 91 65 020604\u202c' : 1,'Enzo' : 1, 'Shriram Yadav' : 1, 'MANDU GJU' : 1,
                     'Kallu' : 1,'\u202a+91\xa096542\xa092717\u202c' : 1, 'The Billionaires Club 🤑' : 1, '\u200eSuyash changed the subject to “Our bitch' : 1,
                     '\u202a+971\xa050\xa0132\xa07301\u202c' : 1,'the complete digital marketing course* - https' : 1, "archu" : 1}},inplace=True)
df.name.unique()

array(['Adamya Rajput', 'Bhandu', 'Meena', 'rajit', 'Sukhman', 'Mayank',
       'Suyash', 'SCOOBY', 1, 'Shady', 'Raman Deora'], dtype=object)

In [0]:
# removing names with value 1

df = df[df["name"] != 1]

In [32]:
df.head()

,date_time,name,text
3,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki
9,2014-01-30 23:05:45,Bhandu,<‎image omitted>
10,2014-01-30 23:05:45,Bhandu,<‎image omitted>
11,2014-01-30 23:05:46,Bhandu,<‎image omitted>
12,2014-01-30 23:05:46,Bhandu,<‎image omitted>


In [0]:
cursing = ["bc" , "mc" , "gand", "chut","chod","lund","laude","rand","cunt","fuck","tutta","choot","bhosd","lavde","bsdk"]

In [0]:
# checking if text has bad words

df['cursing'] = df["text"].str.lower().str.contains('|'.join(cursing))

In [35]:
df.head()

,date_time,name,text,cursing
3,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True
9,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False
10,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False
11,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False
12,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False


In [0]:
multimedia = ["omitted>" , "http" , "www."]

In [0]:
# checking text have some link or is showing default text for media shared

df['multi'] = df["text"].str.lower().str.contains('|'.join(multimedia))

In [38]:
df.head()

,date_time,name,text,cursing,multi
3,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True,False
9,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True
10,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True
11,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True
12,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True


In [56]:
df.to_csv("drive/app/Misc/whatsapp/analysis/clean_text.csv",index=False,encoding='utf-8-sig')

In [58]:
df[df["name"] == "Meena"].shape

(23615, 5)

In [63]:
df[(df["name"] == "Meena") & (df["cursing"] == True)].shape

(2588, 5)